In [1]:
import CropModels
from CropDataset import MyDataSet,normalize_torch,normalize_05,normalize_dataset,preprocess,preprocess_hflip,preprocess_with_augmentation
import pandas as pd
from torch.utils.data import DataLoader
import torch.nn as nn
from tensorboardX import SummaryWriter
import datetime
import os
import torch
from torch.autograd import Variable
from utils import RunningMean
import utils
from PIL import Image
import numpy as np
import random

In [2]:
NB_CLASS=59
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
torch.backends.cudnn.benchmark = True

In [3]:
BATCH_SIZE=32
IMAGE_SIZE=420   # 不同模型修改不同的Size
IMAGE_TRAIN_PRE='../data/AgriculturalDisease_trainingset/images/'
ANNOTATION_TRAIN='../data/AgriculturalDisease_trainingset/AgriculturalDisease_train_annotations_deleteNoise.json' #是否需要剔除两类异常类
IMAGE_VAL_PRE='../data/AgriculturalDisease_validationset/images/'
ANNOTATION_VAL='../data/AgriculturalDisease_validationset/AgriculturalDisease_validation_annotations_deleteNoise.json' #是否需要剔除两类异常类
date=str(datetime.date.today())
with open(ANNOTATION_TRAIN) as datafile1:
    trainDataFram=pd.read_json(datafile1,orient='records')
with open(ANNOTATION_VAL) as datafile2: #first check if it's a valid json file or not
    validateDataFram =pd.read_json(datafile2,orient='records')    
def getmodel():
    print('[+] loading model... ', end='', flush=True)
    model=CropModels.resnet50_finetune(NB_CLASS)
    model.cuda()
    print('Done')
    return model

In [4]:
def train(epochNum):
    writer=SummaryWriter('log/'+date+'/ResNet50/') # 创建 /log/日期/InceptionResnet的组织形式  不同模型需要修改不同名称
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess_with_augmentation(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    weight=torch.Tensor([1,3,3,3,3,4,2,3,3,3,3,3,3,3,3,3,2,3,3,3,2,3,4,2,3,1,1,3,2,2,1,3,3,1,3,2,3,3,3,3,2,1,3,2,3,3,3,1,3,3,4,4,3,2,2,3,1,1,3]).cuda()
    criterion=nn.CrossEntropyLoss(weight=weight).cuda()
#     lx, px = utils.predict(model,val_dataLoader)
#     min_loss = criterion(Variable(px), Variable(lx)).item()
    min_loss=4.1
    print('min_loss is :%f'%(min_loss))
    min_acc=0.80
    patience=0
    lr=0.0
    momentum=0.0
    for epoch in range(epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if epoch==3:
            lr=1e-3
            momentum=0.9
            print('set lr=:%f,momentum=%f'%(lr,momentum))
        if patience==2 and lr==1e-3:
            patience=0
            model.load_state_dict(torch.load('../model/ResNet50/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/10
            print('loss has increased lr divide 10 lr now is :%f'%(lr))
        if patience==2 and lr==1e-4:
            patience=0
            epochNum=epoch+1
        if epoch==0 or epoch==1 or epoch==2: #第一轮首先训练全连接层
            lr=1e-3
#             optimizer=torch.optim.SGD(params=model.fresh_params(),lr=lr,momentum=0.9)
            optimizer = torch.optim.Adam(model.fresh_params(),lr = lr,amsgrad=True,weight_decay=1e-4)
        else:
            optimizer = torch.optim.Adam(model.fresh_params(),lr = lr,amsgrad=True,weight_decay=1e-4)
#             optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    writer.add_scalar('Train/Acc',running_corrects.value,niter)
                    writer.add_scalar('Train/Loss',running_loss.value,niter)
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    writer.add_scalar('Val/Acc',accuracy,niter)
                    writer.add_scalar('Val/Loss',log_loss,niter)
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))       
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        writer.add_scalar('Val/Acc',accuracy,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        writer.add_scalar('Val/Loss',log_loss,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'ResNet50', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'ResNet50', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [5]:
def reuseTrain(path,epochNum):
    writer=SummaryWriter('log/'+date+'/ResNet50/') # 创建 /log/日期/InceptionResnet的组织形式
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess_with_augmentation(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    criterion=nn.CrossEntropyLoss().cuda()
    modelParams=torch.load(path)
    model.load_state_dict(modelParams['state_dict'])
    min_loss=modelParams['val_loss']
    print('min_loss is :%f'%(min_loss))   
    min_acc=max(modelParams['val_correct'],0.80)
    print('val_correct is %f'%(min_acc))
    patience=0
    lr=1e-2
    momentum=0.9
    beginepoch=modelParams['epoch']
    for epoch in range(beginepoch,epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if epoch==beginepoch+2:
            lr=lr/10
        if epoch==beginepoch+4:
            lr=lr/10
        if epoch==beginepoch+6:
            lr=1e-2
        if epoch==beginepoch+8:
            lr=lr/10
        if epoch==beginepoch+10:
            lr=lr/10
#         if patience==3:
#             patience=0
#             model.load_state_dict(torch.load('../model/ResNet50/'+date+'_loss_best.pth')['state_dict'])
#             lr=lr/10

        optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=0.9)
        print('lr now is %f'%(lr))
        print('now patience is %d '%(patience))
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    writer.add_scalar('Train/Acc',running_corrects.value,niter)
                    writer.add_scalar('Train/Loss',running_loss.value,niter)
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    writer.add_scalar('Val/Acc',accuracy,niter)
                    writer.add_scalar('Val/Loss',log_loss,niter)
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))      
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        writer.add_scalar('Val/Acc',accuracy,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        writer.add_scalar('Val/Loss',log_loss,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'ResNet50', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'ResNet50', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [6]:
def TrainWithRawData(path,epochNum):
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    criterion=nn.CrossEntropyLoss().cuda()
    modelParams=torch.load(path)
    model.load_state_dict(modelParams['state_dict'])
    min_loss=modelParams['val_loss']
    print('min_loss is :%f'%(min_loss))
    print('val_correct is %f'%(modelParams['val_correct']))
    min_acc=max(modelParams['val_correct'],0.81)
    optinizerSave=modelParams['optimizer']
    patience=0
    lr=1e-4
    momentum=0.9
    beginepoch=modelParams['epoch']
    for epoch in range(beginepoch,epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if patience==3:
            patience=0
            model.load_state_dict(torch.load('../model/ResNet50/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/5
            print('loss has increased lr divide 10 lr now is :%f'%(lr))
            optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=0.9)
#            optimizer = torch.optim.Adam(model.fresh_params(),lr = lr,amsgrad=True,weight_decay=1e-4)
        
        else:
            optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=0.9)
 #           optimizer = torch.optim.Adam(model.parameters(),lr = lr,amsgrad=True,weight_decay=1e-4)
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
                    if  log_loss < min_loss:
                        utils.snapshot('../model/', 'ResNet50', {
                               'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optimizer': optimizer.state_dict(),
                               'val_loss': log_loss,
                               'val_correct':accuracy })          

                        min_loss=log_loss
                        patience=0
                        print('save new model loss,now loss is ',min_loss)

                    if accuracy>min_acc:
                        utils.snapshot('../model/', 'ResNet50', {
                               'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optimizer': optimizer.state_dict(),
                               'val_loss': log_loss,
                               'val_correct':accuracy },key='acc') 
                        min_acc=accuracy
                        print('save new model acc,now acc is ',min_acc)
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))         
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'ResNet50', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'ResNet50', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [ ]:
train(60)

[+] loading model... Done
min_loss is :4.100000
Epoch 0/59
----------
[epoch:0,batch:29]:acc: 0.155208,loss:5.350667
[epoch:0,batch:59]:acc: 0.251563,loss:4.279010
[epoch:0,batch:89]:acc: 0.323264,loss:3.730664
[epoch:0,batch:119]:acc: 0.364583,loss:3.373821
[epoch:0,batch:149]:acc: 0.388125,loss:3.179203
[epoch:0,batch:179]:acc: 0.409896,loss:3.001765
[epoch:0,batch:209]:acc: 0.426339,loss:2.867028
[epoch:0,batch:239]:acc: 0.442188,loss:2.761445
[epoch:0,batch:269]:acc: 0.458218,loss:2.658466
[epoch:0,batch:299]:acc: 0.469792,loss:2.577482
[epoch:0,batch:299]: val_loss:1.373625,val_acc:0.661599,val_total:4539
[epoch:0,batch:329]:acc: 0.482008,loss:2.498274
[epoch:0,batch:359]:acc: 0.491233,loss:2.436621
[epoch:0,batch:389]:acc: 0.501522,loss:2.369601
[epoch:0,batch:419]:acc: 0.507887,loss:2.318837
[epoch:0,batch:449]:acc: 0.511389,loss:2.287441
[epoch:0,batch:479]:acc: 0.518424,loss:2.245875
[epoch:0,batch:509]:acc: 0.523039,loss:2.208880
[epoch:0,batch:539]:acc: 0.527488,loss:2.19282

[epoch:4,batch:179]:acc: 0.751736,loss:0.767133
[epoch:4,batch:209]:acc: 0.748810,loss:0.769497
[epoch:4,batch:239]:acc: 0.748828,loss:0.766935
[epoch:4,batch:269]:acc: 0.750926,loss:0.752116
[epoch:4,batch:299]:acc: 0.752604,loss:0.741542
[epoch:4,batch:299]: val_loss:0.706182,val_acc:0.762723,val_total:4539
[epoch:4,batch:329]:acc: 0.751326,loss:0.739805
[epoch:4,batch:359]:acc: 0.752604,loss:0.735142
[epoch:4,batch:389]:acc: 0.753205,loss:0.736788
[epoch:4,batch:419]:acc: 0.752753,loss:0.736266
[epoch:4,batch:449]:acc: 0.752153,loss:0.736209
[epoch:4,batch:479]:acc: 0.753060,loss:0.728949
[epoch:4,batch:509]:acc: 0.752880,loss:0.729822
[epoch:4,batch:539]:acc: 0.752141,loss:0.732412
[epoch:4,batch:569]:acc: 0.752632,loss:0.731930
[epoch:4,batch:599]:acc: 0.752552,loss:0.732217
[epoch:4,batch:599]: val_loss:0.692234,val_acc:0.771095,val_total:4539
[epoch:4,batch:629]:acc: 0.753423,loss:0.729741
[epoch:4,batch:659]:acc: 0.754498,loss:0.729474
[epoch:4,batch:689]:acc: 0.754710,loss:0.7

[epoch:8,batch:359]:acc: 0.782899,loss:0.628745
[epoch:8,batch:389]:acc: 0.779968,loss:0.631779
[epoch:8,batch:419]:acc: 0.780432,loss:0.631499
[epoch:8,batch:449]:acc: 0.779931,loss:0.629298
[epoch:8,batch:479]:acc: 0.779818,loss:0.627263
[epoch:8,batch:509]:acc: 0.780331,loss:0.629673
[epoch:8,batch:539]:acc: 0.778935,loss:0.631314
[epoch:8,batch:569]:acc: 0.778783,loss:0.631803
[epoch:8,batch:599]:acc: 0.779635,loss:0.629587
[epoch:8,batch:599]: val_loss:0.600566,val_acc:0.788500,val_total:4539
[epoch:8,batch:629]:acc: 0.780060,loss:0.625765
[epoch:8,batch:659]:acc: 0.780161,loss:0.627323
[epoch:8,batch:689]:acc: 0.780344,loss:0.627738
[epoch:8,batch:719]:acc: 0.779861,loss:0.630321
[epoch:8,batch:749]:acc: 0.779167,loss:0.630839
[epoch:8,batch:779]:acc: 0.778245,loss:0.631288
[epoch:8,batch:809]:acc: 0.777431,loss:0.631725
[epoch:8,batch:839]:acc: 0.778088,loss:0.630356
[epoch:8,batch:869]:acc: 0.777227,loss:0.633833
[epoch:8,batch:899]:acc: 0.776632,loss:0.634301
[epoch:8,batch:89

[epoch:12,batch:599]:acc: 0.789740,loss:0.596409
[epoch:12,batch:599]: val_loss:0.596278,val_acc:0.790482,val_total:4539
[epoch:12,batch:629]:acc: 0.787996,loss:0.597779
[epoch:12,batch:659]:acc: 0.786553,loss:0.600856
[epoch:12,batch:689]:acc: 0.785915,loss:0.602097
[epoch:12,batch:719]:acc: 0.785503,loss:0.602707
[epoch:12,batch:749]:acc: 0.785500,loss:0.603308
[epoch:12,batch:779]:acc: 0.784896,loss:0.604549
[epoch:12,batch:809]:acc: 0.786227,loss:0.600973
[epoch:12,batch:839]:acc: 0.786719,loss:0.599804
[epoch:12,batch:869]:acc: 0.786997,loss:0.598854
[epoch:12,batch:899]:acc: 0.787188,loss:0.597438
[epoch:12,batch:899]: val_loss:0.571981,val_acc:0.799736,val_total:4539
[epoch:12,batch:929]:acc: 0.787366,loss:0.598226
[epoch:12,batch:959]:acc: 0.787533,loss:0.599237
[epoch:12,batch:989]:acc: 0.787563,loss:0.598577
[epoch:12] :acc: 0.787527,loss:0.598044,lr:0.000100,patience:1
[epoch:12]: val_loss:0.581055,val_acc:0.786076,
save new model loss,now loss is  0.5810545086860657
Epoch 1

[epoch:16,batch:719]:acc: 0.795399,loss:0.572921
[epoch:16,batch:749]:acc: 0.795125,loss:0.573486
[epoch:16,batch:779]:acc: 0.794872,loss:0.574640
[epoch:16,batch:809]:acc: 0.795062,loss:0.574205
[epoch:16,batch:839]:acc: 0.795722,loss:0.573456
[epoch:16,batch:869]:acc: 0.795582,loss:0.574631
[epoch:16,batch:899]:acc: 0.795451,loss:0.574498
[epoch:16,batch:899]: val_loss:0.585921,val_acc:0.801718,val_total:4539
[epoch:16,batch:929]:acc: 0.795598,loss:0.574216
[epoch:16,batch:959]:acc: 0.795280,loss:0.575008
[epoch:16,batch:989]:acc: 0.795234,loss:0.575304
[epoch:16] :acc: 0.795252,loss:0.575194,lr:0.000100,patience:1
[epoch:16]: val_loss:0.562471,val_acc:0.803922,
save new model loss,now loss is  0.5624714493751526
save new model acc,now acc is  tensor(0.8039, device='cuda:0')
Epoch 17/10
----------
[epoch:17,batch:29]:acc: 0.787500,loss:0.664852
[epoch:17,batch:59]:acc: 0.787500,loss:0.625431
[epoch:17,batch:89]:acc: 0.788889,loss:0.611409
[epoch:17,batch:119]:acc: 0.783854,loss:0.609

...

[epoch:33,batch:269]:acc: 0.806481,loss:0.529821
[epoch:33,batch:299]:acc: 0.806250,loss:0.527884
[epoch:33,batch:299]: val_loss:0.538163,val_acc:0.801718,val_total:4539
[epoch:33,batch:329]:acc: 0.807292,loss:0.523504
[epoch:33,batch:359]:acc: 0.807639,loss:0.520757
[epoch:33,batch:389]:acc: 0.808333,loss:0.519833
[epoch:33,batch:419]:acc: 0.806845,loss:0.523958
[epoch:33,batch:449]:acc: 0.806944,loss:0.523887
[epoch:33,batch:479]:acc: 0.807878,loss:0.521528
[epoch:33,batch:509]:acc: 0.809130,loss:0.517219
[epoch:33,batch:539]:acc: 0.808044,loss:0.520394
[epoch:33,batch:569]:acc: 0.807182,loss:0.521475
[epoch:33,batch:599]:acc: 0.807969,loss:0.520340
[epoch:33,batch:599]: val_loss:0.521775,val_acc:0.808548,val_total:4539
[epoch:33,batch:629]:acc: 0.808433,loss:0.518832
[epoch:33,batch:659]:acc: 0.808333,loss:0.519956
[epoch:33,batch:689]:acc: 0.808469,loss:0.519240
[epoch:33,batch:719]:acc: 0.807595,loss:0.520474
[epoch:33,batch:749]:acc: 0.808417,loss:0.520226
[epoch:33,batch:779]:ac

[epoch:37,batch:389]:acc: 0.808734,loss:0.515159
[epoch:37,batch:419]:acc: 0.808780,loss:0.515282
[epoch:37,batch:449]:acc: 0.808542,loss:0.516171
[epoch:37,batch:479]:acc: 0.809961,loss:0.510741
[epoch:37,batch:509]:acc: 0.808762,loss:0.512614
[epoch:37,batch:539]:acc: 0.810185,loss:0.510095
[epoch:37,batch:569]:acc: 0.809156,loss:0.511016
[epoch:37,batch:599]:acc: 0.809115,loss:0.509730
[epoch:37,batch:599]: val_loss:0.511766,val_acc:0.810531,val_total:4539
[epoch:37,batch:629]:acc: 0.810417,loss:0.508895
[epoch:37,batch:659]:acc: 0.811032,loss:0.509530
[epoch:37,batch:689]:acc: 0.810417,loss:0.510582
[epoch:37,batch:719]:acc: 0.809722,loss:0.510681
[epoch:37,batch:749]:acc: 0.809333,loss:0.513195
[epoch:37,batch:779]:acc: 0.808814,loss:0.512993
[epoch:37,batch:809]:acc: 0.808333,loss:0.512987
[epoch:37,batch:839]:acc: 0.807850,loss:0.514620
[epoch:37,batch:869]:acc: 0.808836,loss:0.513996
[epoch:37,batch:899]:acc: 0.808715,loss:0.514369
[epoch:37,batch:899]: val_loss:0.575130,val_ac

[epoch:41,batch:599]:acc: 0.815156,loss:0.497835
[epoch:41,batch:599]: val_loss:0.554506,val_acc:0.793567,val_total:4539
[epoch:41,batch:629]:acc: 0.813194,loss:0.501797
[epoch:41,batch:659]:acc: 0.813116,loss:0.502458
[epoch:41,batch:689]:acc: 0.813225,loss:0.502714
[epoch:41,batch:719]:acc: 0.813889,loss:0.502992
[epoch:41,batch:749]:acc: 0.812792,loss:0.504062
[epoch:41,batch:779]:acc: 0.813021,loss:0.504487
[epoch:41,batch:809]:acc: 0.811998,loss:0.505664
[epoch:41,batch:839]:acc: 0.811570,loss:0.507029
[epoch:41,batch:869]:acc: 0.811243,loss:0.506567
[epoch:41,batch:899]:acc: 0.811215,loss:0.506618
[epoch:41,batch:899]: val_loss:0.523980,val_acc:0.813615,val_total:4539
[epoch:41,batch:929]:acc: 0.811727,loss:0.505513
[epoch:41,batch:959]:acc: 0.811751,loss:0.506762
[epoch:41,batch:989]:acc: 0.811458,loss:0.509204
[epoch:41] :acc: 0.811331,loss:0.510289,lr:0.000100,patience:1
[epoch:41]: val_loss:0.573762,val_acc:0.801939,
Epoch 42/40
----------
[epoch:42,batch:29]:acc: 0.803125,lo

[epoch:45,batch:779]:acc: 0.811258,loss:0.498558
[epoch:45,batch:809]:acc: 0.811150,loss:0.497835
[epoch:45,batch:839]:acc: 0.812016,loss:0.497222
[epoch:45,batch:869]:acc: 0.812141,loss:0.496398
[epoch:45,batch:899]:acc: 0.812743,loss:0.496520
[epoch:45,batch:899]: val_loss:0.526196,val_acc:0.807447,val_total:4539
[epoch:45,batch:929]:acc: 0.812836,loss:0.496730
[epoch:45,batch:959]:acc: 0.812760,loss:0.498616
[epoch:45,batch:989]:acc: 0.812279,loss:0.500689
[epoch:45] :acc: 0.812372,loss:0.500298,lr:0.000100,patience:1
[epoch:45]: val_loss:0.527055,val_acc:0.808107,
save new model loss,now loss is  0.5270546078681946
Epoch 46/44
----------
[epoch:46,batch:29]:acc: 0.825000,loss:0.479798
[epoch:46,batch:59]:acc: 0.828125,loss:0.491391
[epoch:46,batch:89]:acc: 0.824306,loss:0.488637
[epoch:46,batch:119]:acc: 0.825260,loss:0.483726
[epoch:46,batch:149]:acc: 0.823125,loss:0.482113
[epoch:46,batch:179]:acc: 0.817361,loss:0.499783
[epoch:46,batch:209]:acc: 0.817857,loss:0.490664
[epoch:46,

[epoch:49,batch:929]:acc: 0.814113,loss:0.504441
[epoch:49,batch:959]:acc: 0.813867,loss:0.504070
[epoch:49,batch:989]:acc: 0.814110,loss:0.503091
[epoch:49] :acc: 0.814138,loss:0.503089,lr:0.000100,patience:1
[epoch:49]: val_loss:0.541961,val_acc:0.802379,
Epoch 50/48
----------
[epoch:50,batch:29]:acc: 0.808333,loss:0.521663
[epoch:50,batch:59]:acc: 0.804167,loss:0.532574
[epoch:50,batch:89]:acc: 0.812500,loss:0.502313
[epoch:50,batch:119]:acc: 0.808594,loss:0.518660
[epoch:50,batch:149]:acc: 0.809583,loss:0.509177
[epoch:50,batch:179]:acc: 0.810069,loss:0.502335
[epoch:50,batch:209]:acc: 0.815923,loss:0.491289
[epoch:50,batch:239]:acc: 0.815625,loss:0.492033
[epoch:50,batch:269]:acc: 0.817245,loss:0.491163
[epoch:50,batch:299]:acc: 0.818229,loss:0.485668
[epoch:50,batch:299]: val_loss:0.530020,val_acc:0.806565,val_total:4539
[epoch:50,batch:329]:acc: 0.819792,loss:0.481528
[epoch:50,batch:359]:acc: 0.818576,loss:0.483799
[epoch:50,batch:389]:acc: 0.817548,loss:0.487223


In [7]:
reuseTrain('../model/ResNet50/2018-11-08_loss_best.pth',90)

[+] loading model... Done
min_loss is :0.383656
val_correct is 0.862084
Epoch 72/89
----------
lr now is 0.010000
now patience is 0 
[epoch:72,batch:29]:acc: 0.782292,loss:0.769916
[epoch:72,batch:59]:acc: 0.615104,loss:1.720353
[epoch:72,batch:89]:acc: 0.569792,loss:1.830606
[epoch:72,batch:119]:acc: 0.577344,loss:1.719513
[epoch:72,batch:149]:acc: 0.592500,loss:1.576185
[epoch:72,batch:179]:acc: 0.614583,loss:1.451779
[epoch:72,batch:209]:acc: 0.633036,loss:1.345298
[epoch:72,batch:239]:acc: 0.645833,loss:1.275229
[epoch:72,batch:269]:acc: 0.656829,loss:1.209314
[epoch:72,batch:299]:acc: 0.666458,loss:1.156987
[epoch:72,batch:299]: val_loss:0.622181,val_acc:0.767350,val_total:4539
[epoch:72,batch:329]:acc: 0.676231,loss:1.109793
[epoch:72,batch:359]:acc: 0.681684,loss:1.075536
[epoch:72,batch:389]:acc: 0.687821,loss:1.040372
[epoch:72,batch:419]:acc: 0.695238,loss:1.004598
[epoch:72,batch:449]:acc: 0.702917,loss:0.972748
[epoch:72,batch:479]:acc: 0.706510,loss:0.952404
[epoch:72,batc

[epoch:76,batch:29]:acc: 0.863542,loss:0.281242
[epoch:76,batch:59]:acc: 0.874479,loss:0.274767
[epoch:76,batch:89]:acc: 0.875347,loss:0.281062
[epoch:76,batch:119]:acc: 0.877083,loss:0.280929
[epoch:76,batch:149]:acc: 0.876250,loss:0.280058
[epoch:76,batch:179]:acc: 0.877951,loss:0.274898
[epoch:76,batch:209]:acc: 0.881101,loss:0.271560
[epoch:76,batch:239]:acc: 0.881641,loss:0.270976
[epoch:76,batch:269]:acc: 0.882176,loss:0.272707
[epoch:76,batch:299]:acc: 0.881250,loss:0.273558
[epoch:76,batch:299]: val_loss:0.353696,val_acc:0.858339,val_total:4539
[epoch:76,batch:329]:acc: 0.881723,loss:0.273409
[epoch:76,batch:359]:acc: 0.882031,loss:0.274045
[epoch:76,batch:389]:acc: 0.883494,loss:0.272527
[epoch:76,batch:419]:acc: 0.883408,loss:0.271289
[epoch:76,batch:449]:acc: 0.883125,loss:0.272703
[epoch:76,batch:479]:acc: 0.883268,loss:0.272826
[epoch:76,batch:509]:acc: 0.883272,loss:0.273290
[epoch:76,batch:539]:acc: 0.883102,loss:0.272599
[epoch:76,batch:569]:acc: 0.882950,loss:0.272332


[epoch:80,batch:89]:acc: 0.876736,loss:0.297236
[epoch:80,batch:119]:acc: 0.875260,loss:0.303819
[epoch:80,batch:149]:acc: 0.880208,loss:0.291964
[epoch:80,batch:179]:acc: 0.882118,loss:0.287732
[epoch:80,batch:209]:acc: 0.879167,loss:0.287935
[epoch:80,batch:239]:acc: 0.879297,loss:0.287737
[epoch:80,batch:269]:acc: 0.879051,loss:0.285446
[epoch:80,batch:299]:acc: 0.878750,loss:0.283851
[epoch:80,batch:299]: val_loss:0.364872,val_acc:0.856356,val_total:4539
[epoch:80,batch:329]:acc: 0.878125,loss:0.286736
[epoch:80,batch:359]:acc: 0.876910,loss:0.286807
[epoch:80,batch:389]:acc: 0.875561,loss:0.286382
[epoch:80,batch:419]:acc: 0.874628,loss:0.287495
[epoch:80,batch:449]:acc: 0.874514,loss:0.286847
[epoch:80,batch:479]:acc: 0.875716,loss:0.285501
[epoch:80,batch:509]:acc: 0.875245,loss:0.285267
[epoch:80,batch:539]:acc: 0.876157,loss:0.283827
[epoch:80,batch:569]:acc: 0.876425,loss:0.283266
[epoch:80,batch:599]:acc: 0.876719,loss:0.282896
[epoch:80,batch:599]: val_loss:0.361217,val_acc

[epoch:84,batch:149]:acc: 0.911458,loss:0.220377
[epoch:84,batch:179]:acc: 0.912847,loss:0.217836
[epoch:84,batch:209]:acc: 0.913393,loss:0.214327
[epoch:84,batch:239]:acc: 0.912630,loss:0.212774
[epoch:84,batch:269]:acc: 0.911806,loss:0.214733
[epoch:84,batch:299]:acc: 0.911771,loss:0.214201
[epoch:84,batch:299]: val_loss:0.372035,val_acc:0.861643,val_total:4539
[epoch:84,batch:329]:acc: 0.911458,loss:0.214872
[epoch:84,batch:359]:acc: 0.909896,loss:0.217726
[epoch:84,batch:389]:acc: 0.909535,loss:0.218778
[epoch:84,batch:419]:acc: 0.909598,loss:0.219495
[epoch:84,batch:449]:acc: 0.909306,loss:0.219630
[epoch:84,batch:479]:acc: 0.909180,loss:0.220253
[epoch:84,batch:509]:acc: 0.909007,loss:0.220358
[epoch:84,batch:539]:acc: 0.908681,loss:0.221009
[epoch:84,batch:569]:acc: 0.908388,loss:0.220769
[epoch:84,batch:599]:acc: 0.909062,loss:0.219857
[epoch:84,batch:599]: val_loss:0.371104,val_acc:0.862084,val_total:4539
[epoch:84,batch:629]:acc: 0.908482,loss:0.220831
[epoch:84,batch:659]:ac

[epoch:88,batch:269]:acc: 0.911343,loss:0.209785
[epoch:88,batch:299]:acc: 0.910625,loss:0.211326
[epoch:88,batch:299]: val_loss:0.380486,val_acc:0.861203,val_total:4539
[epoch:88,batch:329]:acc: 0.908996,loss:0.213431
[epoch:88,batch:359]:acc: 0.909115,loss:0.212269
[epoch:88,batch:389]:acc: 0.908494,loss:0.214077
[epoch:88,batch:419]:acc: 0.907738,loss:0.215324
[epoch:88,batch:449]:acc: 0.907708,loss:0.214403
[epoch:88,batch:479]:acc: 0.907813,loss:0.214537
[epoch:88,batch:509]:acc: 0.907598,loss:0.214085
[epoch:88,batch:539]:acc: 0.907002,loss:0.214436
[epoch:88,batch:569]:acc: 0.906579,loss:0.214549
[epoch:88,batch:599]:acc: 0.906823,loss:0.215458
[epoch:88,batch:599]: val_loss:0.377197,val_acc:0.859661,val_total:4539
[epoch:88,batch:629]:acc: 0.906895,loss:0.215894
[epoch:88,batch:659]:acc: 0.906013,loss:0.217251
[epoch:88,batch:689]:acc: 0.905888,loss:0.217025
[epoch:88,batch:719]:acc: 0.905859,loss:0.216799
[epoch:88,batch:749]:acc: 0.906000,loss:0.216958
[epoch:88,batch:779]:ac

In [7]:
TrainWithRawData('../model/ResNet50/2018-11-09_loss_best.pth',120)

[+] loading model... Done
min_loss is :0.350605
val_correct is 0.859881
Epoch 78/119
----------
[epoch:78,batch:29]:acc: 0.893750,loss:0.253837
[epoch:78,batch:59]:acc: 0.890104,loss:0.260539
[epoch:78,batch:89]:acc: 0.888889,loss:0.266730
[epoch:78,batch:119]:acc: 0.883594,loss:0.273149
[epoch:78,batch:149]:acc: 0.888958,loss:0.262913
[epoch:78,batch:179]:acc: 0.887326,loss:0.263965
[epoch:78,batch:209]:acc: 0.888988,loss:0.261899
[epoch:78,batch:239]:acc: 0.888542,loss:0.263589
[epoch:78,batch:269]:acc: 0.889236,loss:0.262266
[epoch:78,batch:299]:acc: 0.889479,loss:0.262775
[epoch:78,batch:299]: val_loss:0.412199,val_acc:0.854814,val_total:4539
[epoch:78,batch:329]:acc: 0.890530,loss:0.261254
[epoch:78,batch:359]:acc: 0.890538,loss:0.260609
[epoch:78,batch:389]:acc: 0.890064,loss:0.259759
[epoch:78,batch:419]:acc: 0.890402,loss:0.258993
[epoch:78,batch:449]:acc: 0.891111,loss:0.258031
[epoch:78,batch:479]:acc: 0.891276,loss:0.257852
[epoch:78,batch:509]:acc: 0.890809,loss:0.258174
[e

[epoch:82,batch:149]:acc: 0.887708,loss:0.259723
[epoch:82,batch:179]:acc: 0.888021,loss:0.258426
[epoch:82,batch:209]:acc: 0.887649,loss:0.260956
[epoch:82,batch:239]:acc: 0.884505,loss:0.263026
[epoch:82,batch:269]:acc: 0.884722,loss:0.264424
[epoch:82,batch:299]:acc: 0.885417,loss:0.264790
[epoch:82,batch:299]: val_loss:0.395720,val_acc:0.854153,val_total:4539
[epoch:82,batch:329]:acc: 0.886458,loss:0.264074
[epoch:82,batch:359]:acc: 0.886632,loss:0.264641
[epoch:82,batch:389]:acc: 0.887340,loss:0.263419
[epoch:82,batch:419]:acc: 0.887723,loss:0.263058
[epoch:82,batch:449]:acc: 0.887917,loss:0.262170
[epoch:82,batch:479]:acc: 0.887760,loss:0.263073
[epoch:82,batch:509]:acc: 0.888051,loss:0.261746
[epoch:82,batch:539]:acc: 0.887211,loss:0.262430
[epoch:82,batch:569]:acc: 0.887226,loss:0.261373
[epoch:82,batch:599]:acc: 0.887240,loss:0.261532
[epoch:82,batch:599]: val_loss:0.387428,val_acc:0.856576,val_total:4539
[epoch:82,batch:629]:acc: 0.888244,loss:0.260103
[epoch:82,batch:659]:ac

[epoch:86,batch:299]: val_loss:0.412145,val_acc:0.853272,val_total:4539
[epoch:86,batch:329]:acc: 0.886364,loss:0.262945
[epoch:86,batch:359]:acc: 0.887847,loss:0.262643
[epoch:86,batch:389]:acc: 0.888942,loss:0.260857
[epoch:86,batch:419]:acc: 0.889732,loss:0.260855
[epoch:86,batch:449]:acc: 0.889097,loss:0.261477
[epoch:86,batch:479]:acc: 0.887891,loss:0.262699
[epoch:86,batch:509]:acc: 0.888297,loss:0.262223
[epoch:86,batch:539]:acc: 0.888368,loss:0.260964
[epoch:86,batch:569]:acc: 0.889419,loss:0.259225
[epoch:86,batch:599]:acc: 0.888594,loss:0.259110
[epoch:86,batch:599]: val_loss:0.418271,val_acc:0.852831,val_total:4539
[epoch:86,batch:629]:acc: 0.889534,loss:0.257539
[epoch:86,batch:659]:acc: 0.889678,loss:0.257753
[epoch:86,batch:689]:acc: 0.889493,loss:0.257372
[epoch:86,batch:719]:acc: 0.890278,loss:0.256917
[epoch:86,batch:749]:acc: 0.890458,loss:0.257034
[epoch:86,batch:779]:acc: 0.889904,loss:0.257701
[epoch:86,batch:809]:acc: 0.889583,loss:0.257960
[epoch:86,batch:839]:ac

[epoch:90,batch:449]:acc: 0.887708,loss:0.262626
[epoch:90,batch:479]:acc: 0.886458,loss:0.263257
[epoch:90,batch:509]:acc: 0.887377,loss:0.262183
[epoch:90,batch:539]:acc: 0.887095,loss:0.262031
[epoch:90,batch:569]:acc: 0.887555,loss:0.260736
[epoch:90,batch:599]:acc: 0.888177,loss:0.260063
[epoch:90,batch:599]: val_loss:0.396321,val_acc:0.853492,val_total:4539
[epoch:90,batch:629]:acc: 0.888145,loss:0.260986
[epoch:90,batch:659]:acc: 0.888636,loss:0.259974
[epoch:90,batch:689]:acc: 0.888134,loss:0.260374
[epoch:90,batch:719]:acc: 0.888021,loss:0.260361
[epoch:90,batch:749]:acc: 0.887667,loss:0.260698
[epoch:90,batch:779]:acc: 0.887901,loss:0.260559
[epoch:90,batch:809]:acc: 0.887654,loss:0.261582
[epoch:90,batch:839]:acc: 0.887984,loss:0.261181
[epoch:90,batch:869]:acc: 0.888254,loss:0.260920
[epoch:90,batch:899]:acc: 0.887500,loss:0.261859
[epoch:90,batch:899]: val_loss:0.421855,val_acc:0.854373,val_total:4539
[epoch:90,batch:929]:acc: 0.887702,loss:0.261506
[epoch:90,batch:959]:ac

[epoch:94,batch:599]: val_loss:0.386733,val_acc:0.855034,val_total:4539
[epoch:94,batch:629]:acc: 0.885268,loss:0.262494
[epoch:94,batch:659]:acc: 0.885701,loss:0.260753
[epoch:94,batch:689]:acc: 0.886096,loss:0.260717
[epoch:94,batch:719]:acc: 0.886285,loss:0.260646
[epoch:94,batch:749]:acc: 0.886208,loss:0.260536
[epoch:94,batch:779]:acc: 0.885938,loss:0.261433
[epoch:94,batch:809]:acc: 0.886188,loss:0.261222
[epoch:94,batch:839]:acc: 0.886235,loss:0.260594
[epoch:94,batch:869]:acc: 0.886818,loss:0.259971
[epoch:94,batch:899]:acc: 0.887153,loss:0.259165
[epoch:94,batch:899]: val_loss:0.425846,val_acc:0.854814,val_total:4539
[epoch:94,batch:929]:acc: 0.886895,loss:0.260219
[epoch:94,batch:959]:acc: 0.886523,loss:0.260802
[epoch:94,batch:989]:acc: 0.886742,loss:0.260110
[epoch:94] :acc: 0.886748,loss:0.259989,lr:0.000000,patience:1
[epoch:94]: val_loss:0.414167,val_acc:0.853051,
Epoch 95/119
----------
[epoch:95,batch:29]:acc: 0.879167,loss:0.269265
[epoch:95,batch:59]:acc: 0.888542,lo

...

[epoch:102,batch:869]:acc: 0.887751,loss:0.259762
[epoch:102,batch:899]:acc: 0.887604,loss:0.260142
[epoch:102,batch:899]: val_loss:0.390662,val_acc:0.855034,val_total:4539
[epoch:102,batch:929]:acc: 0.887500,loss:0.260280
[epoch:102,batch:959]:acc: 0.887305,loss:0.260422
[epoch:102,batch:989]:acc: 0.887184,loss:0.260510
[epoch:102] :acc: 0.887032,loss:0.260574,lr:0.000000,patience:0
[epoch:102]: val_loss:0.409745,val_acc:0.854594,
Epoch 103/119
----------
[epoch:103,batch:29]:acc: 0.882292,loss:0.281291
[epoch:103,batch:59]:acc: 0.886979,loss:0.266821
[epoch:103,batch:89]:acc: 0.888194,loss:0.263103
[epoch:103,batch:119]:acc: 0.884115,loss:0.263826
[epoch:103,batch:149]:acc: 0.885625,loss:0.264998
[epoch:103,batch:179]:acc: 0.883507,loss:0.266289
[epoch:103,batch:209]:acc: 0.883631,loss:0.269406
[epoch:103,batch:239]:acc: 0.886589,loss:0.264070
[epoch:103,batch:269]:acc: 0.887384,loss:0.261121
[epoch:103,batch:299]:acc: 0.886667,loss:0.260656
[epoch:103,batch:299]: val_loss:0.433419,v

[epoch:106,batch:929]:acc: 0.888138,loss:0.259356
[epoch:106,batch:959]:acc: 0.887793,loss:0.259399
[epoch:106,batch:989]:acc: 0.887563,loss:0.260364
[epoch:106] :acc: 0.887474,loss:0.261588,lr:0.000000,patience:1
[epoch:106]: val_loss:0.418577,val_acc:0.853051,
Epoch 107/119
----------
[epoch:107,batch:29]:acc: 0.891667,loss:0.269128
[epoch:107,batch:59]:acc: 0.886979,loss:0.278767
[epoch:107,batch:89]:acc: 0.889583,loss:0.269595
[epoch:107,batch:119]:acc: 0.890104,loss:0.266519
[epoch:107,batch:149]:acc: 0.888958,loss:0.272147
[epoch:107,batch:179]:acc: 0.889062,loss:0.273830
[epoch:107,batch:209]:acc: 0.890327,loss:0.269727
[epoch:107,batch:239]:acc: 0.890625,loss:0.268965
[epoch:107,batch:269]:acc: 0.890625,loss:0.267839
[epoch:107,batch:299]:acc: 0.891458,loss:0.265727
[epoch:107,batch:299]: val_loss:0.395824,val_acc:0.854814,val_total:4539
[epoch:107,batch:329]:acc: 0.891098,loss:0.266454
[epoch:107,batch:359]:acc: 0.891233,loss:0.265313
[epoch:107,batch:389]:acc: 0.890224,loss:0

[epoch:110] :acc: 0.887820,loss:0.262268,lr:0.000000,patience:2
[epoch:110]: val_loss:0.464799,val_acc:0.849967,
Epoch 111/119
----------
loss has increased lr divide 10 lr now is :0.000000
[epoch:111,batch:29]:acc: 0.883333,loss:0.257457
[epoch:111,batch:59]:acc: 0.886458,loss:0.258588
[epoch:111,batch:89]:acc: 0.889583,loss:0.254153
[epoch:111,batch:119]:acc: 0.893490,loss:0.252008
[epoch:111,batch:149]:acc: 0.889375,loss:0.263131
[epoch:111,batch:179]:acc: 0.888889,loss:0.262898
[epoch:111,batch:209]:acc: 0.890030,loss:0.261092
[epoch:111,batch:239]:acc: 0.889193,loss:0.261049
[epoch:111,batch:269]:acc: 0.891435,loss:0.256767
[epoch:111,batch:299]:acc: 0.890521,loss:0.259030
[epoch:111,batch:299]: val_loss:0.370693,val_acc:0.853933,val_total:4539
[epoch:111,batch:329]:acc: 0.889299,loss:0.261123
[epoch:111,batch:359]:acc: 0.889323,loss:0.260397
[epoch:111,batch:389]:acc: 0.889744,loss:0.258567
[epoch:111,batch:419]:acc: 0.889137,loss:0.260970
[epoch:111,batch:449]:acc: 0.889653,loss

[epoch:115,batch:29]:acc: 0.906250,loss:0.241701
[epoch:115,batch:59]:acc: 0.902604,loss:0.237186
[epoch:115,batch:89]:acc: 0.889583,loss:0.256565
[epoch:115,batch:119]:acc: 0.882031,loss:0.267559
[epoch:115,batch:149]:acc: 0.882917,loss:0.271262
[epoch:115,batch:179]:acc: 0.883333,loss:0.272726
[epoch:115,batch:209]:acc: 0.884673,loss:0.269369
[epoch:115,batch:239]:acc: 0.885286,loss:0.269059
[epoch:115,batch:269]:acc: 0.885185,loss:0.268083
[epoch:115,batch:299]:acc: 0.885938,loss:0.267469
[epoch:115,batch:299]: val_loss:0.384222,val_acc:0.854814,val_total:4539
[epoch:115,batch:329]:acc: 0.886932,loss:0.265883
[epoch:115,batch:359]:acc: 0.886372,loss:0.267627
[epoch:115,batch:389]:acc: 0.885256,loss:0.269389
[epoch:115,batch:419]:acc: 0.884970,loss:0.268367
[epoch:115,batch:449]:acc: 0.884028,loss:0.268788
[epoch:115,batch:479]:acc: 0.884440,loss:0.267237
[epoch:115,batch:509]:acc: 0.884681,loss:0.267359
[epoch:115,batch:539]:acc: 0.884259,loss:0.268647
[epoch:115,batch:569]:acc: 0.8

[epoch:119,batch:119]:acc: 0.888281,loss:0.258984
[epoch:119,batch:149]:acc: 0.891250,loss:0.256048
[epoch:119,batch:179]:acc: 0.888715,loss:0.260054
[epoch:119,batch:209]:acc: 0.886905,loss:0.261515
[epoch:119,batch:239]:acc: 0.887240,loss:0.260043
[epoch:119,batch:269]:acc: 0.886574,loss:0.260045
[epoch:119,batch:299]:acc: 0.886667,loss:0.259486
[epoch:119,batch:299]: val_loss:0.411343,val_acc:0.853492,val_total:4539
[epoch:119,batch:329]:acc: 0.886932,loss:0.257248
[epoch:119,batch:359]:acc: 0.886545,loss:0.257834
[epoch:119,batch:389]:acc: 0.886779,loss:0.256451
[epoch:119,batch:419]:acc: 0.888393,loss:0.253207
[epoch:119,batch:449]:acc: 0.888056,loss:0.253502
[epoch:119,batch:479]:acc: 0.887565,loss:0.254426
[epoch:119,batch:509]:acc: 0.888235,loss:0.254163
[epoch:119,batch:539]:acc: 0.888079,loss:0.254213
[epoch:119,batch:569]:acc: 0.887993,loss:0.253975
[epoch:119,batch:599]:acc: 0.888542,loss:0.253874
[epoch:119,batch:599]: val_loss:0.409454,val_acc:0.852611,val_total:4539
[epo

In [5]:
model=getmodel()

[+] loading model... Done


In [6]:
model

ResNetFinetune(
  (net): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0): Co

In [10]:
m = nn.AdaptiveAvgPool2d(1)
input = torch.randn(1, 64, 8, 9)
m(input)


tensor([[[[ 0.1199]],

         [[-0.0907]],

         [[ 0.0677]],

         [[ 0.0502]],

         [[-0.1503]],

         [[-0.1222]],

         [[-0.0225]],

         [[ 0.1698]],

         [[ 0.1923]],

         [[ 0.0396]],

         [[ 0.0742]],

         [[ 0.2630]],

         [[ 0.0454]],

         [[ 0.2154]],

         [[-0.1502]],

         [[ 0.0754]],

         [[ 0.1026]],

         [[ 0.0296]],

         [[ 0.0801]],

         [[ 0.0084]],

         [[ 0.0664]],

         [[-0.2356]],

         [[ 0.1217]],

         [[ 0.1353]],

         [[ 0.1134]],

         [[-0.0969]],

         [[-0.0584]],

         [[-0.1067]],

         [[-0.3733]],

         [[ 0.0310]],

         [[ 0.1004]],

         [[ 0.0485]],

         [[-0.0359]],

         [[-0.0651]],

         [[-0.0701]],

         [[ 0.0512]],

         [[-0.0109]],

         [[ 0.0539]],

         [[ 0.1152]],

         [[ 0.0118]],

         [[-0.3467]],

         [[ 0.0484]],

         [[-0.1393]],

         [[